In [1]:
// Using Library
// println(java.nio.file.Paths.get("").toAbsolutePath().toString())
// %use @file[[DIR_PATH]/[LIB_NAME].json]

In [2]:
%use krangl
%use coroutines

%use @file[D:/Programming/Kotlin/learn-kotlin/dependencies/gson.json]
%use @file[D:/Programming/Kotlin/learn-kotlin/dependencies/org_json.json]
%use @file[D:/Programming/Kotlin/learn-kotlin/dependencies/retrofit.json]
%use @file[D:/Programming/Kotlin/learn-kotlin/dependencies/retrofit_gson_converter.json]

In [3]:
import kotlinx.coroutines.flow.Flow
import kotlinx.coroutines.flow.flow
import kotlinx.coroutines.flow.MutableStateFlow
import kotlinx.coroutines.flow.StateFlow
import kotlinx.coroutines.flow.collect
import kotlinx.coroutines.GlobalScope
import kotlinx.coroutines.launch

In [4]:
data class UserResponse(
    @SerializedName("incomplete_results")
    val incompleteResults: Boolean?,
    @SerializedName("items")
    val items: List<UserDto>,
    @SerializedName("total_count")
    val totalCount: Long?
)

data class UserDto(
    @SerializedName("id")
    val id: Long,
    @SerializedName("avatar_url")
    val avatarUrl: String?,
    @SerializedName("name")
    val name: String?,
    @SerializedName("login")
    val login: String?,
    @SerializedName("bio")
    val bio: String?,
    @SerializedName("followers")
    val followers: Int?,
    @SerializedName("following")
    val following: Int?,
    @SerializedName("location")
    val location: String?,
    @SerializedName("email")
    val email: String?
)

data class User(
    val id: Long,
    val avatarUrl: String,
    val name: String,
    val login: String,
    val bio: String,
    val followers: Int,
    val following: Int,
    val location: String,
    val email: String
)

In [5]:
fun UserDto.toUser() = User(
    id = id,
    avatarUrl = avatarUrl ?: "",
    name = name ?: "N/A",
    login = login ?: "N/A",
    bio = bio ?: "Biography not available",
    followers = followers ?: 0,
    following = following ?: 0,
    location = location ?: "N/A",
    email = email ?: "N/A"
)

fun List<UserDto>.toListUser() =
    this.map {
        it.toUser()
    }

In [6]:
interface GithubService {
    @JvmSuppressWildcards
    @GET("search/users")
    suspend fun searchUsers(
        @QueryMap params: Map<String, Any>
    ): Response<UserResponse>

    @GET("users/{id}")
    suspend fun getUser(
        @Path("id") username: String
    ): Response<UserDto>
}

In [7]:
val retrofit = Retrofit
        .Builder().apply {
            baseUrl("https://api.github.com/")
            addConverterFactory(GsonConverterFactory.create(GsonBuilder().create()))
        }.build()
        
val service = retrofit.create(GithubService::class.java)

In [8]:
val indramahkota = "{\"login\":\"indramahkota\",\"id\":34052126,\"node_id\":\"MDQ6VXNlcjM0MDUyMTI2\",\"avatar_url\":\"https://avatars.githubusercontent.com/u/34052126?v=4\",\"gravatar_id\":\"\",\"url\":\"https://api.github.com/users/indramahkota\",\"html_url\":\"https://github.com/indramahkota\",\"followers_url\":\"https://api.github.com/users/indramahkota/followers\",\"following_url\":\"https://api.github.com/users/indramahkota/following{/other_user}\",\"gists_url\":\"https://api.github.com/users/indramahkota/gists{/gist_id}\",\"starred_url\":\"https://api.github.com/users/indramahkota/starred{/owner}{/repo}\",\"subscriptions_url\":\"https://api.github.com/users/indramahkota/subscriptions\",\"organizations_url\":\"https://api.github.com/users/indramahkota/orgs\",\"repos_url\":\"https://api.github.com/users/indramahkota/repos\",\"events_url\":\"https://api.github.com/users/indramahkota/events{/privacy}\",\"received_events_url\":\"https://api.github.com/users/indramahkota/received_events\",\"type\":\"User\",\"site_admin\":false,\"name\":\"Indra Mahkota\",\"company\":null,\"blog\":\"indramahkota.info\",\"location\":\"Pontianak, Indonesia\",\"email\":null,\"hireable\":true,\"bio\":\"Focus on developing android applications and having some knowledge of web frontend technologies.\uD83D\uDE0A.\",\"twitter_username\":null,\"public_repos\":54,\"public_gists\":9,\"followers\":20,\"following\":19,\"created_at\":\"2017-11-28T05:26:02Z\",\"updated_at\":\"2022-01-13T07:58:02Z\"}"

In [9]:
val inDto = Gson().fromJson(indramahkota, UserDto::class.java)
val inU = inDto.toUser()
inU

User(id=34052126, avatarUrl=https://avatars.githubusercontent.com/u/34052126?v=4, name=Indra Mahkota, login=indramahkota, bio=Focus on developing android applications and having some knowledge of web frontend technologies.😊., followers=20, following=19, location=Pontianak, Indonesia, email=N/A)

In [10]:
suspend fun searchUsers(params: Map<String, Any>): UserResponse? {
    return try {
        val response = service.searchUsers(params)
        if(response.isSuccessful)
            response.body()
        else null
    } catch (e: Exception) {
        println(e.message ?: "Unknown error")
        null
    }
}

suspend fun getUser(username: String): UserDto? {
    return try {
        val response = service.getUser(username)
        if(response.isSuccessful)
            response.body()
        else null
    } catch (e: Exception) {
        println(e.message ?: "Unknown error")
        null
    }
}

val listOfUser = mutableListOf<User>()
val listUserDto = mutableListOf<UserDto>()

fun searchUsersFunction() = runBlocking {
    val response = searchUsers(mapOf("q" to "josh", "per_page" to 10))
    response?.items?.forEach {
        val userDtoResponse = it.login?.let { username ->
            getUser(username)
        }
        if (userDtoResponse != null) {
            listUserDto.add(userDtoResponse)
        }
    }
    listOfUser.addAll(listUserDto.toListUser())
}

searchUsersFunction()

listOfUser

[User(id=137, avatarUrl=https://avatars.githubusercontent.com/u/137?v=4, name=Joshua Peek, login=josh, bio=Biography not available, followers=2377, following=190, location=San Francisco, CA, email=N/A), User(id=54473, avatarUrl=https://avatars.githubusercontent.com/u/54473?v=4, name=Josh Long, login=joshlong, bio=Josh (@starbuxman) is the Spring Developer Advocate at VMware, an open-source hacker, book/video author and speaker, followers=8173, following=65, location=San Francisco, CA, email=N/A), User(id=6692932, avatarUrl=https://avatars.githubusercontent.com/u/6692932?v=4, name=Joshua Comeau, login=joshwcomeau, bio=Software developer.
Makes stuff., followers=4123, following=25, location=Montreal, QC, email=N/A), User(id=10632, avatarUrl=https://avatars.githubusercontent.com/u/10632?v=4, name=Josh Perez, login=goatslacker, bio=Biography not available, followers=806, following=127, location=Florida, email=N/A), User(id=13760, avatarUrl=https://avatars.githubusercontent.com/u/13760?v=4,

In [11]:
// copy and override value

data class I(
    val name: String
)

val i = I("Saya")
val ii = i.copy(name = "Gue")

println(i.toString())
println(ii.toString())

I(name=Saya)
I(name=Gue)
